
# **Máquina de Turing: Implementação Passo a Passo**
## Introdução

A **Máquina de Turing** é um modelo teórico de computação criado por Alan Turing em 1936.  
Ela é composta por uma fita infinita dividida em células, um ponteiro de leitura/escrita e um conjunto de regras de transição.

### **Componentes principais:**
1. **Fita**: Um conjunto de células que podem conter símbolos.
2. **Ponteiro de Leitura/Escrita**: Um cursor que lê e modifica a fita.
3. **Estados**: A máquina possui um estado atual e segue regras de transição.
4. **Regras de Transição**: Um conjunto de instruções que definem como a máquina reage a diferentes símbolos.

Neste notebook, implementaremos uma **Máquina de Turing simples** passo a passo, explicando cada etapa e testando o funcionamento da máquina conforme avançamos.
    

## **Importando Bibliotecas**

In [1]:
import pandas as pd


## **Criando a Classe da Máquina de Turing**

A Máquina de Turing será representada por uma classe `TuringMachine`.  
Ela possui três elementos principais:
- A **fita** (`tape`): onde os dados são armazenados.
- O **Ponteiro** (`head_position`): que se move para a esquerda ou para a direita.
- O **estado atual** (`state`): que define a ação a ser executada.

Abaixo, definimos essa estrutura.
    

In [2]:
class TuringMachine:
  def __init__(self, tape, initial_position, initial_state):
    self.tape = tape
    self.head_position = initial_position
    self.state = initial_state

### **Teste 1: Criando um objeto da Máquina de Turing**

In [3]:
# Criando uma fita de teste
tape = ['0', '1', 'B', '0', '1']
initial_position = 0
initial_state = 'q0'

machine = TuringMachine(tape, initial_position, initial_state)
machine


## **Visualizando o Estado da Máquina**

Criamos um método para exibir o estado da fita e do Ponteiro em um formato mais intuitivo.
    

In [4]:
class TuringMachine:
  def __init__(self, tape, initial_position, initial_state):
    self.tape = tape
    self.head_position = initial_position
    self.state = initial_state

  def display(self):
    tape_display = "".join(self.tape)
    head_marker = ' ' * self.head_position + '^'
    print(f'{tape_display}\n{head_marker}\nState: {self.state}')

### **Teste 2: Exibindo a fita e a posição do Ponteiro**

In [5]:
machine = TuringMachine(tape, 0, initial_state)
machine.display()

01B01
^
State: q0



## **Movendo o Ponteiro**

Agora, implementamos um método que permite que o Ponteiro se mova para a esquerda (`L`) ou para a direita (`R`).
    

In [6]:
class TuringMachine:
  def __init__(self, tape, initial_position, initial_state):
    self.tape = tape
    self.head_position = initial_position
    self.state = initial_state

  def display(self):
    tape_display = "".join(self.tape)
    head_marker = ' ' * self.head_position + '^'
    print(f'{tape_display}\n{head_marker}\nState: {self.state}')

  def move_head(self, one_direction):
    if one_direction == 'L':
      if self.head_position > 0:
        self.head_position -= 1
    elif one_direction == 'R':
      self.head_position += 1

### **Teste 3: Movendo o Ponteiro para a direita e para a esquerda**

In [7]:
machine = TuringMachine(tape, initial_position, initial_state)
machine.display()

print("Movendo para a direita")
machine.move_head('R')
machine.display()

print("Movendo para a esquerda")
machine.move_head('L')
machine.display()


01B01
^
State: q0
Movendo para a direita
01B01
 ^
State: q0
Movendo para a esquerda
01B01
^
State: q0



## **Definindo as Regras de Transição**

Cada regra define:
- O símbolo lido na fita.
- O estado atual da máquina.
- O símbolo que será escrito na fita.
- O movimento do Ponteiro (`L` para esquerda, `R` para direita).
- O novo estado após a transição.

O programa será carregado a partir de um arquivo CSV.
    

### **Fita Inicial**
A fita contém a seguinte sequência de símbolos:
```
['0', '1', 'B', '0', '1']
```
Cada célula pode conter um símbolo, e a máquina inicia a leitura a partir do primeiro símbolo (`0`).

### **Objetivo da Máquina**
A Máquina de Turing percorre a fita e substitui `0` e `1` por `_` até encontrar um espaço em branco (`B`). Quando encontra `B`, escreve `0` se no estado `q0` e `1` se no estado `q1`, move-se para a esquerda e entra no estado final `qf`.

### **Regras de Transição**
A máquina segue as seguintes regras:

| Símbolo Atual | Estado Atual | Novo Símbolo | Movimento | Novo Estado |
|--------------|-------------|-------------|-----------|------------|
| 0            | q0          | _           | R         | q1         |
| 0            | q1          | _           | R         | q0         |
| 1            | q0          | _           | R         | q0         |
| 1            | q1          | _           | R         | q1         |
| B            | q0          | 0           | L         | qf         |
| B            | q1          | 1           | L         | qf         |

### **Resultado Esperado**
Após a execução, a fita deve conter:
```
['_', '_', '1', '0', '1']
```

In [8]:
# Carregar o programa via arquivo CSV
program = pd.read_csv('./turing-machine-example-program.csv', delimiter=';')

# Exibir as regras do programa
print("Regras de transição carregadas:")
display(program)

Regras de transição carregadas:


,symbol,state,write,move,new-state
0,0,q0,_,R,q1
1,0,q1,_,R,q0
2,1,q0,_,R,q0
3,1,q1,_,R,q1
4,B,q0,0,L,qf
5,B,q1,1,L,qf
6,B,qf,NaN,NaN,NaN



## **Execução da Máquina de Turing**

Agora, implementamos um método para executar a máquina até atingir um estado final.
    

In [9]:
def run_turing_machine(machine, program):
    while machine.state != 'qf':
        current_symbol = machine.tape[machine.head_position]
        current_state = machine.state

        # Busca a regra de transição para o símbolo e estado atual
        rule = program[(program['symbol'] == current_symbol) & (program['state'] == current_state)]

        if rule.empty:
            print("Nenhuma transição encontrada. Máquina encerrada.")
            break

        # Aplicando as regras
        new_symbol = rule.iloc[0]['write']
        move_direction = rule.iloc[0]['move']
        new_state = rule.iloc[0]['new-state']

        # Atualiza o símbolo na fita
        machine.tape[machine.head_position] = new_symbol

        # Move o ponteiro conforme a direção especificada
        if move_direction == 'R':
            machine.move_head('R')
        elif move_direction == 'L':
            machine.move_head('L')

        # Atualiza o estado
        machine.state = new_state
        machine.display()
 

machine = TuringMachine(tape, initial_position, initial_state)
run_turing_machine(machine, program)


_1B01
 ^
State: q1
__B01
  ^
State: q1
__101
 ^
State: qf


Uma implementação mais simples...

In [10]:
def run_turing_machine_2(machine):
    for item in machine.tape:
        
        # mostrando a máquina a cada iteração
        machine.display()

        # verificando se está no estado final
        if machine.state == 'qf':
            break

        # regra 1
        if item == '0' and machine.state == 'q0':
            machine.tape[machine.head_position] = '_'
            machine.move_head('R')
            machine.state = 'q1'
            continue

        # regra 2
        if item == '0' and machine.state == 'q1':
            machine.tape[machine.head_position] = '_'
            machine.move_head('R')
            machine.state = 'q0'
            continue

        # regra 3
        if item == '1' and machine.state == 'q0':
            machine.tape[machine.head_position] = '_'
            machine.move_head('R')
            machine.state = 'q0'
            continue

        # regra 4
        if item == '1' and machine.state == 'q1':
            machine.tape[machine.head_position] = '_'
            machine.move_head('R')
            machine.state = 'q1'
            continue

        # regra 5
        if item == 'B' and machine.state == 'q0':
            machine.tape[machine.head_position] = '0'
            machine.move_head('L')
            machine.state = 'qf'
            continue
        
        # regra 6
        if item == 'B' and machine.state == 'q1':
            machine.tape[machine.head_position] = '1'
            machine.move_head('L')
            machine.state = 'qf'
            continue


# inicializando uma máquina
tape = ['0', '1', 'B', '0', '1']
initial_position = 0
initial_state = 'q0'
machine = TuringMachine(tape, initial_position, initial_state)

run_turing_machine_2(machine)


__B01
  ^
State: q0
__B01
  ^
State: q0
__001
 ^
State: qf
__001
 ^
State: qf
__101
 ^
State: qf


# **Exercício:**
    
Agora que vimos como funciona uma Máquina de Turing, implemente uma nova versão que realize uma operação diferente.  

## **Objetivo**  
Crie uma Máquina de Turing que **inverta todos os bits** em uma fita binária. Ou seja:  
- `0` deve ser transformado em `1` e muda de estado
- `1` deve ser transformado em `0` e muda de estado
- O Ponteiro deve percorrer toda a fita e parar quando encontrar um espaço em branco (`B`).  

## **Passo a Passo**  
1. **Defina uma nova fita de entrada**, como: `['0', '1', '1', '0', 'B']`.  
2. **Crie uma tabela de transição** que inverta os valores:  
   - Se `0` for encontrado no estado `q0`, transforme em `1`, mova para a direita e muda para o estado `q1`. Se no estado `q1`, mude para `q0`
   - Se `1` for encontrado no estado `q0`, transforme em `0`, mova para a direita e mude o estado para `q1`. Se no estado `q1`, mude para `q0`
   - Se `B` for encontrado, vai para estado final e pare a execução.  
3. **Implemente a lógica da máquina**, modificando a classe `TuringMachine`.  
4. **Execute a máquina e verifique o resultado esperado:**  

Implemente a sua solução no código abaixo:
    

1. Qual a saída esperada?

In [11]:
tape = ['0', '1', '1', '0', 'B']
initial_position = 0
initial_state = 'qf'

# 1 1 1 0 B
#   ^
# q0 -> q1

# 1 0 1 0 B
#     ^
# q1 -> q0

# 1 0 0 0 B
#       ^
# q0 -> q1

# 1 0 0 1 B
#         ^
# q1 -> q0

# 1 0 0 1 B
#         ^
# q0 -> qf

# expected output:

print("""
1001B
    ^
State: qf
""")


1001B
    ^
State: qf



2. Implemente a sua solução nas células abaixo:

In [12]:
tabela = {
    'symbol':    ['0', '0', '1', '1', 'B', 'B'],
    'state':     ['q0', 'q1', 'q0', 'q1', 'q0', 'q1'],
    'write':     ['1', '1', '0', '0', 'B', 'B'],
    'move':      ['R', 'R', 'R', 'R', '', ''],
    'new-state': ['q1', 'q0', 'q1', 'q0', 'qf', 'qf']
}
program = pd.DataFrame(tabela)
program

,symbol,state,write,move,new-state
0,0,q0,1,R,q1
1,0,q1,1,R,q0
2,1,q0,0,R,q1
3,1,q1,0,R,q0
4,B,q0,B,,qf
5,B,q1,B,,qf


In [13]:
tape = ['0', '1', '1', '0', 'B']
initial_position = 0
initial_state = 'q0'

machine = TuringMachine(tape, initial_position, initial_state)

run_turing_machine(machine, program)

1110B
 ^
State: q1
1010B
  ^
State: q0
1000B
   ^
State: q1
1001B
    ^
State: q0
1001B
    ^
State: qf
